In [1]:
import pandas as pd
import numpy as np
import sys
import pickle
import glob
import os

In [2]:
import peakutils

In [3]:
NUMBER_OF_RUNS_IN_EXPERIMENT = 20

In [1]:
merged_df = pd.read_pickle('/Users/darylwilding-mcbride/Downloads/experiments/190719_Hela_Ecoli/training-sets/merged_df.pkl')

NameError: name 'pd' is not defined

Generate the training sets for each run

In [ ]:
mapping_df = pd.read_pickle('/Users/darylwilding-mcbride/Downloads/percolator_mapping_df.pkl')

In [ ]:
TRAINING_SET_DIR = '/Users/darylwilding-mcbride/Downloads/experiments/190719_Hela_Ecoli/training-sets'

In [ ]:
for file_idx in range(NUMBER_OF_RUNS_IN_EXPERIMENT):
    X_df = merged_df[merged_df.file_idx == file_idx][['theoretical_mz','experiment_average_scan','experiment_average_rt']]
    y_df = merged_df[merged_df.file_idx == file_idx][['run_average_mz','run_average_scan','run_average_rt']]
    run_name = mapping_df[mapping_df.file_idx == file_idx].run_name.iloc[0]
    training_set_file_name_X = "{}/X_df-file-{}-{}.pkl".format(TRAINING_SET_DIR, file_idx, run_name)
    X_df.to_pickle(training_set_file_name_X)
    training_set_file_name_y = "{}/y_df-file-{}-{}.pkl".format(TRAINING_SET_DIR, file_idx, run_name)
    y_df.to_pickle(training_set_file_name_y)

Extract the training set for file_idx = 0

In [ ]:
X_df = merged_df[merged_df.file_idx == 0][['theoretical_mz','experiment_average_scan','experiment_average_rt']]
y_df = merged_df[merged_df.file_idx == 0][['run_average_mz','run_average_scan','run_average_rt']]

In [ ]:
X = X_df.values
y = y_df.values

In [ ]:
from sklearn import preprocessing

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
X_scaled = min_max_scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X_scaled, y, test_size=0.3)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5)

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, Dense

In [ ]:
keras.backend.clear_session()

In [ ]:
inputs = Input(shape=(3,), dtype='float32', name='main_input')

In [ ]:
x = Dense(32, activation='relu')(inputs)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(32, activation='relu')(x)
predictions = Dense(3, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=inputs, outputs=predictions)

In [ ]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train,
          batch_size=8, epochs=200,
          validation_data=(X_val, y_val))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f, ax1 = plt.subplots()
f.set_figheight(8)
f.set_figwidth(15)

plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

In [ ]:
model.evaluate(X_test, y_test)[1]

In [ ]:
model.predict(X_train)

In [ ]:
X_train

In [ ]:
model.predict_on_batch(X_test)